# Analyzing results

In [1]:
import sys,os
root_path = os.path.abspath(os.path.join('..'))
print(root_path)
if root_path not in sys.path:
    sys.path.append(root_path)
%run -i "../lib/utils/utils.py"
%load_ext autoreload
%autoreload 2
%matplotlib inline

/Users/ens/repos/marl


## Computing Triples

### Get a number of games

In [ ]:
start_index = 0
num_games = 100
path = "../data/diplomacy-v1-27k-msgs/standard_no_press.jsonl"
games_jsons = load_jsonl(path, num_games=num_games, mmap=False, completed_only=True)

In [ ]:
# Convert to a pandas dataframe
df = pd.DataFrame(games_jsons)

In [ ]:
games = df['phases']
games.apply(lambda x: x[0])

In [ ]:
for game in games:
    for ix, iy in enumerate(game):
        game[ix]['phase_id'] = ix

### Flatten json records

In [ ]:
all_records = []
for idx, game in enumerate(games):
    for idx, phase in enumerate(game):
        row_generator = flatten_json(phase)
        assert row_generator is not None, row_generator
        for row in row_generator:
            all_records.append(row.copy())

### Get the complete dataframe with an added column

In [ ]:
complete_df = pd.DataFrame.from_records(all_records)
complete_df['unique_unit_id'] = -1

### Get the unique games in the dataframe

In [ ]:
unique_games = complete_df["game_id"].unique()

### Do temporal binding of units

In [ ]:
game_phase_df_list = []
for idx, game_id in enumerate(unique_games):
    phases_df_list = []
    print(idx, game_id, flush=True)
    s_dict = {}
    d_dict = {}
    _id = 1
    game_df = complete_df.loc[complete_df["game_id"].apply(lambda x: x == game_id)]
    unique_phases = game_df['phase_id'].unique()
    for phase in unique_phases:
        condition = game_df["phase_id"].apply(lambda x: x == phase)
        phase_df = game_df.loc[condition]
        s_dict, d_dict, _id = assign_unit_id(phase_df, s_dict, d_dict, _id, discard_disband_creation=False)
        phases_df_list.append(phase_df)
    phases_cdf = pd.concat(phases_df_list)
    dislodged_df = phases_cdf.loc[phases_cdf['action'] == -2].copy()
    replace_dislodged_units(phases_cdf, dislodged_df)
    game_phase_df_list.append(phases_cdf)


In [ ]:
cdf = pd.concat(game_phase_df_list)

### Assert no invalid rows are left

In [ ]:
assert cdf.loc[cdf['coordinator'] == 'RA'].empty

### Filter out adjustment and retreat seasons

In [ ]:
spring_fall_phases=(cdf['phase_name'].apply(lambda x:x[0])!='W') & (cdf['phase_name'].apply(lambda x:x[-1])!='R') & (cdf['phase_name'].apply(lambda x:x[-1]) == 'M')
cdf_sf = cdf.loc[spring_fall_phases].copy()
cdf_sf['phase_num']=cdf_sf.phase_name.apply(lambda x: float(x[1:-1]+('.0' if x[0]=='S' else '.5')))

### Get the triples' presence based on the filtered dataframe

In [ ]:
game_tiple_presence = {}
for idx, game_id in enumerate(unique_games):
    assert type(game_id) is str, (game_id, "is not a string")
    print(idx, game_id, flush=True)
    game_df = cdf_sf.loc[cdf_sf['game_id'] == game_id]
    
    if game_df.unique_unit_id.nunique() == game_df.unique_unit_id.max():
        triples = get_triples(game_df)
        try:
            emp = get_triples_presence(game_df, triples)
            print(emp, len(triples), flush=True)
            game_tiple_presence[game_id] = triples
        except AssertionError as msg:
            print(msg, flush=True)

### Save json file if necessary

In [ ]:
    name = "whatever"
    with open(f'{name}.json', 'w') as file:          
        json.dump(game_tiple_presence, file, indent=4, sort_keys=True,)

## Analyze the gathered triples

In [2]:
path = ""
with open(f'{path}', 'r') as file:          
    game_tiple_presence = json.load(file)

In [3]:
game_tiple_presence.keys()

dict_keys(['rW_Q9TpidGLGD2EL'])

In [4]:
list(game_tiple_presence.values())[0][0]

{'max_min_diff': 1.5,
 'max_phase_num': 1902.5,
 'min_phase_num': 1901.0,
 'player_i': 'AUSTRIA',
 'player_j': 'ITALY',
 'triple': [1, 2, 13]}

In [7]:
len(game_tiple_presence)

1

In [12]:
all_records = []
for row in gen_triple_rows(game_tiple_presence):
    all_records.append(row.copy())

In [15]:
df = pd.DataFrame.from_records(all_records)

In [17]:
df.head()

,game_id,player_i,player_j,min_phase_num,max_phase_num,triple
0,rW_Q9TpidGLGD2EL,AUSTRIA,ITALY,1901.0,1902.5,"[1, 2, 13]"
1,rW_Q9TpidGLGD2EL,AUSTRIA,ITALY,1901.0,1902.5,"[1, 2, 14]"
2,rW_Q9TpidGLGD2EL,AUSTRIA,ITALY,1901.0,1902.5,"[1, 2, 15]"
3,rW_Q9TpidGLGD2EL,AUSTRIA,ITALY,1902.0,1902.5,"[1, 2, 29]"
4,rW_Q9TpidGLGD2EL,AUSTRIA,ITALY,NaN,NaN,"[1, 2, 41]"
